In [21]:
#Intro to Web Scraping and BeautifulSoup
#https://www.youtube.com/watch?v=XQgXKtPSzUI

import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re #for splitting strings using multiple delimiters

names = []
prices = []
saleprices = []
soldouts = []
stores = []

In [3]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []

#Pearl River Exotics
URL = 'https://www.pearlriverexotics.com/collections/nepenthes'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
page_count = soup.find('li',attrs={'class':'pagination__text'}).text.strip()
page_count = page_count[len(page_count)-1:] #grabs how many pages to loop through
page_count = int(page_count)

for page_number in range(1,1+page_count):
    URL = 'https://www.pearlriverexotics.com/collections/nepenthes?page=%d' % page_number
    page = urlopen(URL)
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('a',attrs={'class':'grid-view-item__link'})

    for container in containers:
        name_box = container.find('div',attrs={'class':'h4 grid-view-item__title'})
        name = name_box.text.strip()

        price_box = container.find('span',attrs={'class':'product-price__price'})
        price = price_box.text.strip()
        price = price[1:] #remove the 1st character of the string '$'
    
        names = names + [name]
        prices = prices + [price]
        stores = stores + ['Pearl River Exotics']
        saleprices = saleprices + [''] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!change when on sale
        
        #looks for sold out items
        try:
            soldout = container.find('span',attrs={'class':'product-price__sold-out'}).text.strip()
            soldouts = soldouts + [soldout.lower()]
        except AttributeError:
            soldouts = soldouts + ['']
            

d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores}
df = pd.DataFrame(d)

In [4]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []

#California Carnivores
highland_neps = 'https://www.californiacarnivores.com/collections/highland-tropical-pitcher-plants'
lowland_neps = 'https://www.californiacarnivores.com/collections/lowland-nepenthes'
sections = [highland_neps,lowland_neps]

for page_number in range(2):
    #select page number
    URL = sections[page_number]
    page = urlopen(URL)
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half'})
    containers_sale = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half on-sale'})
    
    #grabs information of normal items
    for container in containers:
        name_box = container.find('p',attrs={'class':'grid-link__title'})
        name = name_box.text.strip()

        price_box = container.find('p',attrs={'class':'grid-link__meta'})
        price = price_box.text.strip()
        separate_index = price.find('$') #look for 1st instance of $
        price = price[separate_index+2:] #remove the all characters before and including the string '$ '
    
        names = names + [name]
        prices = prices + [price]
        saleprices = saleprices + ['']
        stores = stores + ['California Carnivores']
        soldouts = soldouts + ['']
        
    #grabs information of sale items
    for container in containers_sale:
        name_box = container.find('p',attrs={'class':'grid-link__title'})
        name = name_box.text.strip()

        price_box = container.find('p',attrs={'class':'grid-link__meta'})
        price = price_box.text.strip() #price of items on sale should be of form '$ xx.xx\n$ xx.xx'
        separate_index = price.rfind('$') #from end of string, look for 1st instance of $
        price_before = price[separate_index:] #gets original price
        price_before = price_before[2:] #remove the first 2 characters of the string '$ '
        price_after = price[:separate_index] #gets sale price
        price_after = price_after[2:] #remove the first 2 characters of the string '$ '
        price_after = price_after[:-1] #remove the last 2 characters of the string '$ '
    
        names = names + [name]
        prices = prices + [price_before]
        saleprices = saleprices + [price_after]
        stores = stores + ['California Carnivores']
        soldouts = soldouts + ['']
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)

In [5]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []

#Cook's Carnivorous Plants
for page_number in range(19,25):
    #select page number: nepenthes species highland(19) to nepenthes hybrid lowland(25)
    URL = 'http://www.flytraps.com/Scripts/prodList.asp?idCategory=%d' % page_number
    page = urlopen(URL)
    soup = BeautifulSoup(page,'html.parser')
    
    containers = soup.findAll('h3')
    containers_count = len(containers)

    #names of items
    for i in range(1,containers_count): #omitting the first container
        name_box = soup.findAll('h3')[i]
        name = name_box.text
        names = names + [name]
        stores = stores + ['Cook\'s Carnivorous Plants']
        soldouts = soldouts + ['']
    
    #prices of items
    for i in range(containers_count - 1):
        price_box = soup.findAll('del',attrs={'class':'CPprodLPriceV'})[i]
        price = price_box.text
        separate_index = price.find('$') #look for 1st instance of $
        price = price[separate_index+1:] #remove the all characters before and including the string '$'
        prices = prices + [price]
    
    #prices of items on sale
    containers_count_sale = len(soup.findAll('span',attrs={'class':'price'}))
    for i in range(containers_count_sale):
        price_box = soup.findAll('span',attrs={'class':'price'})[i]
        price = price_box.text
        separate_index = price.find('$') #look for 1st instance of $
        price = price[separate_index+1:] #remove the all characters before and including the string '$'
        price = price[:-3] #remove ' ea' from end of sales price
        saleprices = saleprices + [price]    
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)

In [6]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []

#Carnivero
URL = 'https://www.carnivero.com/collections/tropicals-pitcher-plants-nepenthes'
page_next = True

while page_next:
    page = urlopen(URL)
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('div',attrs={'class':'product grid__item medium-up--one-third small--one-half slide-up-animation animated'})
    
    for container in containers:
        name_box = container.find('div',attrs={'class':'product__title text-center'})
        name = name_box.text.strip()

        try:
            price_box = container.find('span',attrs={'class':'product__price'})
            price = price_box.text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+1:] #remove the all characters before and including the string '$'
            prices = prices + [price]
            saleprices = saleprices + ['']           
        except AttributeError:
            #grab original price
            price_box = container.find('s')
            price = price_box.text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+1:] #remove the all characters before and including the string '$'
            prices = prices + [price]
            
            #grab new sales price
            price_box = container.find('span',attrs={'class':'product__price--on-sale'})
            price = price_box.text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+1:] #remove the all characters before and including the string '$'
            saleprices = saleprices + [price] 
            
        try:
            soldout = container.find('strong',attrs={'class':'sold-out-text'}).text.strip()
            soldouts = soldouts + [soldout.lower()]
        except AttributeError:
            soldouts = soldouts + ['']
            
        names = names + [name]
        stores = stores + ['Carnivero']
        
    #Updates the URL for next page; stops the while loop if there is no next page
    try:
        URL = soup.find('span',attrs={'class':'next'})
        URL = URL.a['href']
        URL = 'https://www.carnivero.com' + URL
    except AttributeError:
        page_next = False
        
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)      

In [7]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []

#Native Exotics
URL = 'https://nativeexoticsonline.com/product-category/nepenthes/'
h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
page_next = True

while page_next:
    req = Request(URL,headers=h) 
    page = urlopen(req).read() 
    soup = BeautifulSoup(page,'html.parser')
    container_big = soup.find('ul',attrs={'class':'products columns-4'})
    containers = container_big.findAll('li')
    
    for container in containers:
        name = container.find('h2').text.strip()
        
        price = container.find('span',attrs={'class':'price'}).text.strip()
        price = price[2:] 
        
        try:
            soldout = container.find('span',attrs={'class':'soldout'}).text.strip()
            soldouts = soldouts + [soldout.lower()]
        except AttributeError:
            soldouts = soldouts + ['']
            
        names = names + [name]
        prices = prices + [price]
        stores = stores + ['Native Exotics']
        saleprices = saleprices + [''] #no sales at Native Exotics
        
    #Updates the URL for next page; stops the while loop if there is no next page
    try:
        URL = soup.find('a',attrs={'class':'next page-numbers'})
        URL = URL['href']
    except TypeError:
        page_next = False
        
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)    

In [8]:
#Once the complete database is compiled, apply tagging function - tags any species found in product
#https://www.carnivorousplants.co.uk/resources/nepenthes-interactive-guide/
#https://en.wikipedia.org/wiki/List_of_Nepenthes_species

#compare product to database of species known
#if str in database, then species = str

In [11]:
n = 172
name_nepenthes = [''] * n
URL = 'https://en.wikipedia.org/wiki/List_of_Nepenthes_species'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
containers = soup.findAll('tr')

for i in range(1,n):
    container = containers[i]
    name = container.find('a')
    name_nepenthes[i] = name['title'][10:] #removing 'Nepenthes ' from string
    
name_nepenthes = name_nepenthes[1:] #remove header row
name_nepenthes

['abalata',
 'abgracilis',
 'adnata',
 'aenigma',
 'alata',
 'alba',
 'albomarginata',
 'alzapan',
 'ampullaria',
 'andamana',
 'angasanensis',
 'appendiculata',
 'argentii',
 'aristolochioides',
 'armin',
 'attenboroughii',
 'barcelonae',
 'beccariana',
 'bellii',
 'benstonei',
 'biak',
 'bicalcarata',
 'bokorensis',
 'bongso',
 'boschiana',
 'burbidgeae',
 'burkei',
 'campanulata',
 'ceciliae',
 'chang',
 'chaniana',
 'cid',
 'clipeata',
 'copelandii',
 'cornuta',
 'danseri',
 'deaniana',
 'densiflora',
 'diatas',
 'distillatoria',
 'dubia',
 'edwardsiana',
 'ephippiata',
 'epiphytica',
 'eustachya',
 'extincta',
 'eymae',
 'faizaliana',
 'flava',
 'fusca',
 'gantungensis',
 'glabrata',
 'glandulifera',
 'graciliflora',
 'gracilis',
 'gracillima',
 'gymnamphora',
 'halmahera',
 'hamata',
 'hamiguitanensis',
 'hemsleyana',
 'hirsuta',
 'hispida',
 'holdenii',
 'hurrelliana',
 'inermis',
 'insignis',
 'izumiae',
 'jacquelineae',
 'jamban',
 'junghuhnii',
 'justinae',
 'kampotiana',
 'k

In [38]:
df['Cross1'] = ''
df['Cross2'] = ''
df['Cross3'] = ''
df['Cross4'] = ''
df['Cross5'] = ''
df['Cross6'] = ''

#i is row index, j is column index
#len(df) gives number of rows in df
for i in range(len(df)):
    string_split = df.at[i,'Name'].replace('(','').replace(')','').replace('[','').replace(']','').replace(',','')
    #removing all of the special characters:()[] so the string is split into 'ventricosa' and not '(ventricosa''
    string_split = string_split.split(' ')
    count = 1                           #counter for column name
    for j in range(len(string_split)):
        if string_split[j] in name_nepenthes:
            column = 'Cross%d' % count
            count = count + 1
            df.loc[[i],[column]] = string_split[j]

            
            

#also consider the cases for natural/common hybrids: ventrata, miranda
#create special if clause when tag is not found, open product page and scan description text for tags
df

,Name,Price,Sale Price,Sold Out,Store,Cross1,Cross2,Cross3,Cross4,Cross5,Cross6,Cross7
0,72+ Hour Heat Pack - For shipping with plants,4.00,,,Pearl River Exotics,,,,,,,
1,Nepenthes burbidgeae BE-3834 (SEED-GROWN),59.00,,,Pearl River Exotics,burbidgeae,,,,,,
2,Nepenthes burbidgeae x aristolochioides BE-3784,29.00,,,Pearl River Exotics,burbidgeae,aristolochioides,,,,,
3,Nepenthes burbidgeae x campanulata BE-3564,35.00,,sold out,Pearl River Exotics,burbidgeae,campanulata,,,,,
4,Nepenthes burkei x veitchii BE-3934,16.00,,,Pearl River Exotics,burkei,veitchii,,,,,
5,Nepenthes eymae BE-3736,30.00,,,Pearl River Exotics,eymae,,,,,,
6,Nepenthes glabrata x hamata - BE-3912,45.00,,,Pearl River Exotics,glabrata,hamata,,,,,
7,Nepenthes glandulifera BE-3766 *Seed-grown*,30.00,,,Pearl River Exotics,glandulifera,,,,,,
8,Nepenthes glandulifera BE-3879 (Single best cl...,30.00,,,Pearl River Exotics,glandulifera,,,,,,
9,Nepenthes lingulata BE-3463,59.00,,,Pearl River Exotics,lingulata,,,,,,


In [39]:
#Check what does not have any Cross1 tag
for i in range(len(df)):
    if df.at[i,'Cross1'] is '':
        print(df.at[i,'Name'])

72+ Hour Heat Pack - For shipping with plants
Nepenthes x Miranda *ROOTED CUTTING*
Nepenthes x Red Leopard *SPECIMEN*
Utricularia humboldtii x alpina
72 Hour Heat Pack
Nepenthes klossi (S-TC) Potted
72 Hour Heat Pack
Nepenthes Lady Luck
Nepenthes "Suki"
Nepenthes hookeriana BE-3737
Nepenthes tentactulata, BE-3870
Nepenthes copellandii
Nepenthes x Trusmadiensis "Phil Mann"
High Quality Live Sphagnum Moss – Red/Green – Native Exotics
Nepenthes carunculata – Wistuba – Rare
Nepenthes carunculata – Wistuba – Rare – Specimen
Heliamphora sarracenioides – Ptari – Tepui, Venezuela – Wistuba – Mature – Very Rare – Specimen
N. izumeae x TM – EP
Nepenthes oblanceolata – Seed Grown – Christian Klein – Very Rare – Specimen
Nepenthes Poi Dog – Leilani Nepenthes – Hawaii
Nepenthes sp. G. Murud – Seed Grown – Christian Klein – Specimen
Nepenthes sp. G. Murud – Seed Grown – Christian Klein – Specimen
Nepenthes sp. G. Murud – Seed Grown – Christian Klein – Specimen #2
Nepenthes sp. nov. Papua New Guinea 

In [ ]:
subset = df[df['store']=='Bergen Water Gardens']
subset['price'].mean()

In [ ]:
table = pd.pivot_table(df, values='price', columns=['cross1'], aggfunc=np.mean)
table

In [36]:
df.at[0,'Cross1'] is ''

True